In [8]:
!pip install gcloud


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/twisted-23.10.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 154.0 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=725f0b2d00fe5cfef28511677ba2947f48d3b9ed40f41afc82a799170a944333
  Stored in directory: /Users/rishit/Library/Caches/pip/wheels/3c/e8/d1/cb82a63f69083ea485de71d14248b8d145f1af46a41578be9c
Successfully built gcloud
DEPRECATION: colab 1.13.5 has a non-standard dependency specifier pytz>=2011n. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of colab or contact the author to suggest that they release a version with a conforming dependenc

In [10]:
import ee
import geemap

# Authenticate and initialize Earth Engine API
ee.Initialize(project= 'ee-rishitchugh')

# Define a region of interest (India in this case)
india = ee.FeatureCollection("projects/ee-rishitchugh/assets/boundaryindia")
region = india.geometry()

# Define forest cover dataset
forest_cover = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")

# Define active fire dataset
active_fire = ee.ImageCollection('MODIS/006/MOD14A1').select('FireMask')

# Filter active fire by region and date
active_fire_region = active_fire.filterBounds(region).filterDate('2024-01-01', '2024-12-31')

# Get forest outlines
forest_outlines = forest_cover.select('treecover2000').clip(region).gt(30)

# Display the forest outlines
Map = geemap.Map()
Map.centerObject(region, 5)
Map.addLayer(forest_outlines, {'palette': ['green']}, 'Forest Outlines')
Map.addLayer(active_fire_region, {'palette': ['red']}, 'Active Fires')
Map.addLayerControl()
Map

Map(center=[23.18013323802498, 79.48173585050104], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
import ee
import geemap

# Authenticate and initialize Earth Engine API
ee.Initialize()

# Define a region of interest (India in this case)
india_boundary = ee.FeatureCollection("projects/ee-rishitchugh/assets/boundaryindia")

# Define forest cover dataset
forest_cover = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")

# Define active fire dataset
active_fire = ee.ImageCollection('MODIS/006/MOD14A1').select('FireMask')

# Filter active fire by region and date
active_fire_region = active_fire.filterBounds(india_boundary).filterDate('2024-01-01', '2024-12-31')

# Get forest outlines within India boundary
forest_outlines = forest_cover.select('treecover2000').clip(india_boundary).gt(30)

# Display the forest outlines
Map = geemap.Map()
Map.centerObject(india_boundary, 5)
Map.addLayer(forest_outlines, {'palette': ['green']}, 'Forest Outlines')
Map.addLayer(india_boundary, {'color': 'FF0000'}, 'India Boundary')
Map.addLayer(active_fire_region, {'palette': ['red']}, 'Active Fires')
Map.addLayerControl()
Map

Map(center=[23.18013323802498, 79.48173585050104], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:


# Define the polygon coordinates
polygon = ee.Geometry.Polygon(
    [[[-122.092, 37.424],
      [-122.086, 37.418],
      [-122.079, 37.425],
      [-122.085, 37.431]]])

# Load an image collection
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(polygon) \
    .filterDate('2022-01-01', '2022-12-31')

# Select the first image from the collection
image = ee.Image(collection.first())

# Display the image
Map = geemap.Map()
Map.centerObject(polygon, 12)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'High-Definition Image')
Map.addLayer(polygon, {'color': 'FF0000'}, 'Polygon')
Map.addLayerControl()
Map


EEException: Image.visualize: Parameter 'image' is required.

In [25]:
import ee
import requests
from zipfile import ZipFile
import os
from PIL import Image
import numpy as np

# Authenticate and initialize Earth Engine
ee.Initialize()

# Define the area of interest (AOI) using the provided coordinates
polygon_coords = [[92.66307988555883, 22.766064486620838],
                  [92.75337376983617, 22.71540304165518],
                  [92.81620183380102, 22.772395846595987],
                  [92.77877965362524, 22.84866562418858],
                  [92.72316136749242, 22.8635348344712],
                  [92.66307988555883, 22.766064486620838]]

polygon = ee.Geometry.Polygon(polygon_coords)

# Load Sentinel-2 imagery
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(polygon) \
    .filterDate('2022-01-01', '2022-12-31') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20)

# Select bands
sentinel2_rgb = sentinel2.median().visualize(bands=['B4', 'B3', 'B2'], max=2000)

# Get download URL
download_url = sentinel2_rgb.getDownloadURL({
    'scale': 10,  # adjust scale for desired resolution
    'region': polygon
})

# Download the image
response = requests.get(download_url)
zip_file_path = 'forest_image.zip'

# Save the downloaded file to a local folder
with open(zip_file_path, 'wb') as f:
    f.write(response.content)

# Unzip the downloaded file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('local_folder')

# Remove the zip file
os.remove(zip_file_path)

# Combine separate bands into RGB image
red_band_path = 'local_folder/download.vis-red.tif'
green_band_path = 'local_folder/download.vis-green.tif'
blue_band_path = 'local_folder/download.vis-blue.tif'

red_band = Image.open(red_band_path)
green_band = Image.open(green_band_path)
blue_band = Image.open(blue_band_path)

# Convert to numpy arrays
red_arr = np.array(red_band)
green_arr = np.array(green_band)
blue_arr = np.array(blue_band)

# Stack bands to create RGB image
rgb_image = np.dstack((red_arr, green_arr, blue_arr))

# Convert to PIL image
rgb_image_pil = Image.fromarray(rgb_image)

# Save as PNG
png_file_path = 'local_folder/forest_image.png'
rgb_image_pil.save(png_file_path)

# Remove the individual band files
os.remove(red_band_path)
os.remove(green_band_path)
os.remove(blue_band_path)

In [24]:
import ee
import requests
from zipfile import ZipFile
import os
from PIL import Image
import numpy as np

# Authenticate and initialize Earth Engine
ee.Initialize()

# Define the area of interest (AOI) using the provided coordinates
polygon_coords = [[92.66307988555883, 22.766064486620838],
                  [92.75337376983617, 22.71540304165518],
                  [92.81620183380102, 22.772395846595987],
                  [92.77877965362524, 22.84866562418858],
                  [92.72316136749242, 22.8635348344712],
                  [92.66307988555883, 22.766064486620838]]

polygon = ee.Geometry.Polygon(polygon_coords)

# Load Landsat 8 imagery
landsat8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(polygon) \
    .filterDate('2022-01-01', '2022-12-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 60)  # Adjust the threshold here

# Select bands
landsat8_rgb = landsat8.median().visualize(bands=['B4', 'B3', 'B2'], max=3000)

# Get download URL
download_url = landsat8_rgb.getDownloadURL({
    'scale': 10,  # adjust scale for desired resolution
    'region': polygon
})

# Download the image
response = requests.get(download_url)
zip_file_path = 'forest_image.zip'

# Save the downloaded file to a local folder
with open(zip_file_path, 'wb') as f:
    f.write(response.content)

# Unzip the downloaded file
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('local_folder')

# Remove the zip file
os.remove(zip_file_path)

# Combine separate bands into RGB image
red_band_path = 'local_folder/download.vis-red.tif'
green_band_path = 'local_folder/download.vis-green.tif'
blue_band_path = 'local_folder/download.vis-blue.tif'

red_band = Image.open(red_band_path)
green_band = Image.open(green_band_path)
blue_band = Image.open(blue_band_path)

# Convert to numpy arrays
red_arr = np.array(red_band)
green_arr = np.array(green_band)
blue_arr = np.array(blue_band)

# Stack bands to create RGB image
rgb_image = np.dstack((red_arr, green_arr, blue_arr))

# Convert to PIL image
rgb_image_pil = Image.fromarray(rgb_image)

# Save as PNG
png_file_path = 'local_folder/forest_image.png'
rgb_image_pil.save(png_file_path)

# Remove the individual band files
os.remove(red_band_path)
os.remove(green_band_path)
os.remove(blue_band_path)


In [38]:
import ee
import requests
from zipfile import ZipFile
import os
from PIL import Image
import numpy as np

# Authenticate and initialize Earth Engine
ee.Initialize()

# Define the area of interest (AOI) using the provided coordinates
polygon_coords = [
 [75.62599747394907,13.396101569027806],
 [75.64073889469492,13.395642344018412],
 [75.6411680481373 ,13.405202392891226],
 [75.62556832050669,13.405411123342931],
 [75.62599747394907,13.396101569027806],
]

polygon = ee.Geometry.Polygon(polygon_coords)

# Load Sentinel-2 imagery with reduced cloud cover
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(polygon) \
    .filterDate('2022-04-01', '2022-7-30') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)  # Adjust the threshold here

# Get a list of images in the collection
image_list = sentinel2.toList(sentinel2.size())

# Iterate over each image and download
for i in range(image_list.size().getInfo()):
    # Get the image
    image = ee.Image(image_list.get(i))
    
    # Select bands
    sentinel2_rgb = image.visualize(bands=['B4', 'B3', 'B2'], max=2000)
    
    # Get download URL
    download_url = sentinel2_rgb.getDownloadURL({
        'scale': 10,  # adjust scale for desired resolution
        'region': polygon
    })

    # Download the image
    response = requests.get(download_url)
    zip_file_path = f'forest_image_{i}.zip'  # Unique name for each image

    # Save the downloaded file to a local folder
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)

    # Unzip the downloaded file
    with ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(f'local_folder/{i}')  # Extract to a unique folder for each image

    # Remove the zip file
    os.remove(zip_file_path)


In [39]:


for i in range(1):
    red_band_path = 'local_folder/'+str(i)+'/download.vis-red.tif'
    green_band_path = 'local_folder/'+str(i)+'/download.vis-green.tif'
    blue_band_path = 'local_folder/'+str(i)+'/download.vis-blue.tif'

    red_band = Image.open(red_band_path)
    green_band = Image.open(green_band_path)
    blue_band = Image.open(blue_band_path)

    # Convert to numpy arrays
    red_arr = np.array(red_band)
    green_arr = np.array(green_band)
    blue_arr = np.array(blue_band)

    # Stack bands to create RGB image
    rgb_image = np.dstack((red_arr, green_arr, blue_arr))

    # Convert to PIL image
    rgb_image_pil = Image.fromarray(rgb_image)

    # Save as PNG
    png_file_path = 'local_folder/'+str(i)+'/forest_image.png'
    rgb_image_pil.save(png_file_path)

    # Remove the individual band files
    os.remove(red_band_path)
    os.remove(green_band_path)
    os.remove(blue_band_path)